<h1 style=font:garmond;font-size:50px;color:orange;text-align:left>Predict ML</h1>

In [1]:
import csv
import numpy as np
import pandas as pd

In [2]:
column_names = ['A', 'B', 'C', 'D', 'E', 'F']
df = pd.read_csv('data.csv', names=column_names)

print(df)

       A   B   C   D   E   F
0      7   8  11  20  25  32
1      5  13  19  21  28  42
2      4  10  18  35  36  38
3      8  16  17  23  29  32
4      4   7  10  16  27  41
...   ..  ..  ..  ..  ..  ..
2062  16  19  29  34  37  40
2063   3   9  14  24  34  41
2064   4   6  11  12  34  40
2065   5  16  30  33  35  39
2066   7  15  18  27  31  41

[2067 rows x 6 columns]


<h2 style=font:garmond;font-size:36px;color:orange;text-align:left>Transformation</h2>

In [3]:
from sklearn.preprocessing import StandardScaler

In [4]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_dataframe = pd.DataFrame(data=transformed_dataset, index=df.index)

<h2 style=font:garmond;font-size:36px;color:orange;text-align:left>Modeling</h2>

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional
from keras.optimizers import SGD

In [ ]:
number_of_rows = df.values.shape[0]
number_of_features = df.values.shape[1]
window_size = 30

In [ ]:
train = np.empty([number_of_rows - window_size, window_size, number_of_features], dtype=float)
label = np.empty([number_of_rows - window_size, number_of_features], dtype=float)

for i in range(0, number_of_rows - window_size):
    train[i] = transformed_dataframe.iloc[i:i + window_size, 0: number_of_features]
    label[i] = transformed_dataframe.iloc[i + window_size: i + window_size + 1, 0: number_of_features]

print(f'{train.shape}\n{label.shape}')

In [ ]:
model = Sequential()
model.add(LSTM(1400, input_shape=(window_size, number_of_features), return_sequences=True))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(700, input_shape=(window_size, number_of_features), return_sequences=False)))
model.add(Dense(number_of_features, activation='softmax'))

In [ ]:
learning_rate = 0.11
momentum = 0.4
decay_rate = 0.001

sgd = SGD(learning_rate=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)

In [ ]:
model.compile(loss='mse', optimizer=sgd, metrics='accuracy')

In [ ]:
batch_size = pow(number_of_features, 2)
epochs = 2000

history = model.fit(train, label, batch_size=batch_size, epochs=epochs)

In [ ]:
model.summary()

<h2 style=font:garmond;font-size:36px;color:orange;text-align:left>Result</h2>

In [ ]:
to_predict = np.array(df.iloc[-window_size:])
scaled_to_predict = scaler.transform(to_predict)

scaled_prediction = model.predict(np.array([scaled_to_predict]))
np_prediction = scaler.inverse_transform(scaled_prediction).astype(int)[0]

print(*np_prediction, sep=', ')

In [ ]:
with open('storage.csv', 'a', encoding='UTF-8', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(np_prediction)

<h2 style=font:garmond;font-size:36px;color:orange;text-align:left>Post-training Analysis</h2>

In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
column_names = ['A', 'B', 'C', 'D', 'E', 'F']
df2 = pd.read_csv('storage.csv', names=column_names)

print(df2)

In [ ]:
today = datetime.today()
now = datetime.now()

date_formatted = today.strftime('%m/%d/%y')
time_formatted = now.strftime('%H:%M')

title: str = f'{date_formatted}-{time_formatted}'

plt.figure(figsize=(19,9))
plt.plot(history.history['accuracy'])
plt.title(title)
plt.ylabel('accuracy', fontsize=14)
plt.xlabel('epoch', fontsize=14)
plt.grid()
plt.show()